In [12]:
singularity exec -B /lustre1,/staging,/data,/vsc-hard-mounts,/scratch /data/leuven/software/biomed/singularity_images/scenicplus/scenicplus.sif ipython3

In [1]:
outDir = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/scenicplus_final_autoreg/grnboost/'
import dill
infile = open(outDir+'scplus_obj.pkl', 'rb')
scplus_obj = dill.load(infile)
infile.close()

In [2]:
import pandas as pd
import tqdm
from tqdm import tqdm
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor

SKLEARN_REGRESSOR_FACTORY = {
    'RF': RandomForestRegressor,
    'ET': ExtraTreesRegressor,
    'GBM': GradientBoostingRegressor
}

GBM_KWARGS = {
    'learning_rate': 0.01,
    'n_estimators': 500,
    'max_features': 0.1
}
def train_gene_expression_models_df(X, df, regressor_type = 'GBM', regressor_kwargs = GBM_KWARGS):
    regressors = {}
    for gene in tqdm(set(df['Gene']), total = len(set(df['Gene']))):
        regressor = SKLEARN_REGRESSOR_FACTORY[regressor_type](**regressor_kwargs)
        predictor_TF = list(set(df.query("Gene == @gene")["TF"]))
        if len(predictor_TF) == 0:
            continue
        predictor_TF_exp_v = X[predictor_TF].to_numpy()
        if len(predictor_TF_exp_v.shape) == 1:
            predictor_TF_exp_v = predictor_TF_exp_v.reshape(-1, 1)
        predictand_target_gene_exp_v = X[gene].to_numpy()
        regressor.fit(predictor_TF_exp_v, predictand_target_gene_exp_v)
        regressors[gene] = [*predictor_TF, regressor]
    return regressors

In [30]:
from sklearn.metrics import r2_score
from tqdm import tqdm
df = scplus_obj.to_df('EXP')
X = df.sample(frac = 0.8)
Y = df.drop(X.index)

path = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/methods_benchmark/plots/panelg/df_prediction/'
genes = list(set(list(pd.read_csv(path+'DEGs.tsv', sep='\t', header=None)[0])))
methods = ['CellOracle', 'FigR', 'Pando', 'GRaNIE', 'SCENIC(v10)', 'SCENIC(v10c)', 'SCENIC(v10u)', 'SCENIC(v9)', 'SCENIC+(S)', 'SCENIC+'] 
for method in methods:
    print(method)
    file = path+method+'.tsv'
    d = pd.read_csv(file, sep='\t')
    d = d[d['Gene'].isin(genes)]
    regressors = train_gene_expression_models_df(X, d)
    r2_scores = [
        r2_score(Y[gene].to_numpy(), regressors[gene][-1].predict(Y[regressors[gene][0: len(regressors[gene]) - 1]]))
        for gene in tqdm(regressors.keys(), total = len(regressors.keys()))]
    y = pd.DataFrame([regressors.keys(), r2_scores]).T
    y.to_csv(path+'predictions_v2/'+method+'.tsv',sep='\t', index=False, header=False)

  6%|█████████▏                                                                                                                                                            | 48/872 [00:12<03:26,  3.99it/s]Exception ignored in: <generator object tqdm.__iter__ at 0x2ad365fd6350>
Traceback (most recent call last):
  File "/opt/venv/lib/python3.8/site-packages/tqdm/std.py", line 1210, in __iter__
    self.close()
  File "/opt/venv/lib/python3.8/site-packages/tqdm/std.py", line 1316, in close
    self.display(pos=0)
  File "/opt/venv/lib/python3.8/site-packages/tqdm/std.py", line 1509, in display
    self.sp(self.__str__() if msg is None else msg)
  File "/opt/venv/lib/python3.8/site-packages/tqdm/std.py", line 350, in print_status
    fp_write('\r' + s + (' ' * max(last_len[0] - len_s, 0)))
  File "/opt/venv/lib/python3.8/site-packages/tqdm/std.py", line 344, in fp_write
    fp_flush()
  File "/opt/venv/lib/python3.8/site-packages/tqdm/utils.py", line 145, in inner
    return func(*args, **

KeyboardInterrupt: 

In [3]:
import pandas as pd
import tqdm
from tqdm import tqdm
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor

SKLEARN_REGRESSOR_FACTORY = {
    'RF': RandomForestRegressor,
    'ET': ExtraTreesRegressor,
    'GBM': GradientBoostingRegressor
}

GBM_KWARGS = {
    'learning_rate': 0.01,
    'n_estimators': 500,
    'max_features': 0.1
}
def train_gene_expression_models_df(X, df, regressor_type = 'GBM', regressor_kwargs = GBM_KWARGS):
    regressors = {}
    for gene in tqdm(set(df['Gene']), total = len(set(df['Gene']))):
        regressor = SKLEARN_REGRESSOR_FACTORY[regressor_type](**regressor_kwargs)
        predictor_TF = list(set(df.query("Gene == @gene")["TF"]))
        if len(predictor_TF) == 0:
            continue
        predictor_TF_exp_v = X[predictor_TF].to_numpy()
        if len(predictor_TF_exp_v.shape) == 1:
            predictor_TF_exp_v = predictor_TF_exp_v.reshape(-1, 1)
        predictand_target_gene_exp_v = X[gene].to_numpy()
        regressor.fit(predictor_TF_exp_v, predictand_target_gene_exp_v)
        regressors[gene] = [*predictor_TF, regressor]
    return regressors

In [4]:
from sklearn.metrics import r2_score
from tqdm import tqdm
df = scplus_obj.to_df('EXP')
X = df.sample(frac = 0.8)
Y = df.drop(X.index)

path = '/staging/leuven/stg_00002/lcb/cbravo/Multiomics_pipeline/analysis/DPCL/methods_benchmark/plots/panelg/df_prediction/'
genes = ['FABP1']
methods = ['CellOracle', 'FigR', 'Pando', 'GRaNIE', 'SCENIC(v10)', 'SCENIC(v10c)', 'SCENIC(v10u)', 'SCENIC(v9)', 'SCENIC+(S)', 'SCENIC+'] 
for method in methods:
    print(method)
    file = path+method+'.tsv'
    d = pd.read_csv(file, sep='\t')
    d = d[d['Gene'].isin(genes)]
    regressors = train_gene_expression_models_df(X, d)
    gene = genes[0]
    y = pd.DataFrame([Y[gene].to_numpy(), regressors[gene][-1].predict(Y[regressors[gene][0: len(regressors[gene]) - 1]])]).T
    y.columns = ['O', 'E']
    y.to_csv(path+'predictions_v2/FABP1/'+method+'.tsv',sep='\t', index=False, header=False)

CellOracle


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.06it/s]


FigR


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.85it/s]


Pando


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.63it/s]


SCENIC+(S)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.14s/it]


SCENIC+


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.10it/s]


{0.0, 1.0, 2.0, 3.0, 4.0, 5.0}

In [26]:
y = pd.DataFrame([list(d['Gene']), r2_scores]).T

In [29]:
method = 'CellOracle'
y.to_csv(path+'predictions/'+method+'.tsv',sep='\t', index=False, header=False)

In [4]:
import scanpy as sc
adata = sc.AnnData(scplus_obj.to_df('EXP'))
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

hvg = list(adata.var.index[adata.var['highly_variable']])
genes_to_use = list(set([*hvg, * [x.split('_')[0] for x in scplus_obj.uns['eRegulons']['Gene_based']]]))

regressors = train_gene_expression_models(
        scplus_obj,
        eRegulon_metadata_key = 'eRegulon_metadata_filtered',
        genes = genes_to_use,
        eRegulons_to_use = scplus_obj.uns['selected_eRegulons']['Gene_based'])

X = scplus_obj.to_df('EXP')
from sklearn.metrics import r2_score
from tqdm import tqdm
r2_scores = [
        r2_score(X[gene].to_numpy(), regressors[gene][-1].predict(X[regressors[gene][0: len(regressors[gene]) - 1]]))
        for gene in tqdm(regressors.keys(), total = len(regressors.keys()))]

KeyError: 'selected_eRegulons'

In [7]:
len(hvg)

4308